In [1]:
import numpy as np
import pandas as pd
import mysql.connector
from ast import literal_eval # 문자형 딕셔너리 변환


In [2]:
data = pd.read_csv('ted_talks.csv')
data = pd.DataFrame(data)
data = data.where(pd.notnull(data), None)


CREATE TABLE IF NOT EXISTS `tedbear`.`related_talks` (

  `id` INT NOT NULL AUTO_INCREMENT,
  
  `talks_id` INT NOT  NULL,
  
  `related_id` INT NOT  NULL,
  
  `related_title` TEXT(100000) NULL,
  
  PRIMARY KEY (`id`))
  
ENGINE = InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;



In [3]:
a=list()
b=list()
for i in range(data.shape[0]):
    c=literal_eval(data['related_talks'][i])
    for j in range(0,len(c.keys())):
        a.append(list(c.keys())[j])
        b.append(list(c.values())[j])
d=list()
for j in range(0,len(data)):
    talk_id=data['talk_id'][j]
    c=literal_eval(data['related_talks'][j])
    for i in range(0,len(c)):
        d.append(talk_id)

In [4]:
related=pd.DataFrame(columns=['talks_id','related_id','related_title'])
related['talks_id']=d
related['related_id']=a
related['related_title']=b

In [5]:
related

,talks_id,related_id,related_title
0,1,243,New thinking on the climate crisis
1,1,547,The business logic of sustainability
2,1,2093,The state of the climate — and what we might d...
3,1,64693,The new urgency of climate change
4,1,54715,How we can turn the tide on climate
...,...,...,...
24537,65280,21650,The myth of Hercules
24538,65280,27674,The myth of Sisyphus
24539,65280,49279,The myth of the Sampo— an infinite source of f...
24540,65280,50986,"The Greek myth of Talos, the first robot"


In [6]:


mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    port = 3306,
    password = "1234",
    database = "tedbear"
)
mycursor = mydb.cursor()


In [7]:
cols = "`,`".join([str(i) for i in related.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in related.iterrows():
    sql = "INSERT INTO related_talks (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    mycursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
mydb.commit()

In [8]:
sql = "select * from related_talks"
mycursor.execute(sql)

In [9]:
rows = mycursor.fetchall()
print(rows)     # 전체 r

[(1, 1, 243, 'New thinking on the climate crisis'), (2, 1, 547, 'The business logic of sustainability'), (3, 1, 2093, 'The state of the climate — and what we might do about it'), (4, 1, 64693, 'The new urgency of climate change'), (5, 1, 54715, 'How we can turn the tide on climate'), (6, 1, 60080, "How we could change the planet's climate future"), (7, 2, 1561, 'Energy from floating algae pods'), (8, 2, 1072, "Using nature's genius in architecture"), (9, 2, 1184, 'Cooking as never seen before'), (10, 2, 1406, 'Inventing is the easy part. Marketing takes work'), (11, 2, 767, 'Innovating to zero!'), (12, 3, 127, 'Want to help Africa? Do business here'), (13, 3, 1929, 'We need money for aid. So let’s print it.'), (14, 3, 584, 'New rules for rebuilding a broken nation'), (15, 3, 270, 'The "bottom billion"'), (16, 3, 625, 'Post-crash, investing in a better world'), (17, 3, 152, 'Aid versus trade'), (18, 4, 141, "Inside the world's deepest caves"), (19, 4, 264, 'The astonishing hidden world 

In [10]:
mycursor.close()
mydb.close()